In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sb

In [3]:
train_data = pd.read_csv('/content/drive/MyDrive/Credit_Risk2/Train_main_1Mil.csv')
test_data1 = pd.read_csv('/content/drive/MyDrive/Credit_Risk2/Test1_1Mil.csv')
test_data2 = pd.read_csv('/content/drive/MyDrive/Credit_Risk2/Test2_1Mil.csv')

In [4]:
X_train = train_data.drop(["target","S_2","customer_ID"], axis = 1)
Y_train = train_data["target"]

X_test1 = test_data1.drop(["target","S_2","customer_ID"], axis = 1)
Y_test1 = test_data1["target"]

X_test2 = test_data2.drop(["target","S_2","customer_ID"], axis = 1)
Y_test2 = test_data2["target"]

In [5]:
#Feature importance for test2
import xgboost as xgb
#Setting a default XGBoost classifier
xgb_instance = xgb.XGBClassifier(random_state=100) # nothing inside paranthesis, meaning we are using default parameters, with 100 trees.

model_for_feature_selection = xgb_instance.fit(X_train, Y_train)

feature_importance = {'Feature':X_test2.columns,'Importance':model_for_feature_selection.feature_importances_}
feature_importance = pd.DataFrame(feature_importance)
feature_importance.sort_values("Importance", inplace=True,ascending=False)
feature_importance
final_features1 = feature_importance[feature_importance.Importance > 0.005]
final_features1.to_csv("/content/drive/MyDrive/Credit_Risk2/Final_featuresdefault.csv")

params = {
    'n_estimators': 300,
    'learning_rate': 0.5,
    'max_depth': 4,
    'subsample': 0.5,
    'colsample_bytree': 0.5,
    'scale_pos_weight': 5,
    'random_state':100,
}

xgb_instance = xgb.XGBClassifier(**params)
model_for_feature_selection = xgb_instance.fit(X_train, Y_train)

feature_importance = {'Feature':X_test2.columns,'Importance':model_for_feature_selection.feature_importances_}
feature_importance = pd.DataFrame(feature_importance)
feature_importance.sort_values("Importance", inplace=True,ascending=False)
feature_importance
final_features2 = feature_importance[feature_importance.Importance > 0.005]
final_features2.to_csv("/content/drive/MyDrive/Credit_Risk2/Final_featurescustom.csv")

In [5]:
final_features =["B_7","P_2","B_3","B_38","D_45","D_48","S_7","R_1","D_42","D_66","B_11","B_18","D_132","R_27","S_3","D_43","B_39","B_9","D_79","D_112","D_50","D_51","B_1","B_2","D_75","D_44","O","D_41","D_62","S_23","B_5","B_4","P_3","D_46","S_24","D_49","B_6","D_39"
]
X_train = X_train[final_features]
X_test1 = X_test1[final_features]
X_test2 = X_test2[final_features]

In [7]:
import xgboost as xgb
from sklearn.metrics import roc_auc_score
table = pd.DataFrame(columns = ["Num Trees", "Learning Rate","SS","CS","scale" ,"AUC Train", "AUC Test1","AUC Test2"])
#100 0.1 0.5 0.5 10
#table values Nt300LR0.1SS0.8CS1.0scale10
#300	0.1	0.5	1	5	0.9498304495	0.9245171633	0.9537603347
#for num_trees in [300]:
 # for LR in [0.01, 0.1]:
    
 #   for SS in [0.5,0.8]:
#      for CS in [0.5,1]:
 #       for scale in [1, 5, 10]:
row = 0
for num_trees in [300]:
  for LR in [0.1]:
    
    for SS in [0.8]:
      for CS in [0.5,1]:
        for scale in [1,5,10]:
          xgb_instance = xgb.XGBClassifier(n_estimators=num_trees, learning_rate = LR,subsample = SS,colsample_bytree = CS,scale_pos_weight =scale,random_state =100,
                                   verbosity = 2)
          model = xgb_instance.fit(X_train, Y_train )
          #print (num_trees,LR,SS,CS,scale)
          table.loc[row,"Num Trees"] = num_trees
          table.loc[row,"Learning Rate"] = LR
          table.loc[row,"SS"] = SS
          table.loc[row,"CS"] = CS
          table.loc[row,"scale"] = scale
          table.loc[row,"AUC Train"] = roc_auc_score(Y_train, model.predict_proba(X_train)[:,1])
          table.loc[row,"AUC Test1"] = roc_auc_score(Y_test1, model.predict_proba(X_test1)[:,1])
          table.loc[row,"AUC Test2"] = roc_auc_score(Y_test2, model.predict_proba(X_test2)[:,1])
          print (num_trees,LR,SS,CS,scale)

          row = row + 1
          table.to_csv(r"/content/drive/MyDrive/Credit_Risk2/table values Nt" + str(num_trees) + "LR" + str(LR) + "SS" + str(SS) + "CS" + str(CS) + "scale" + str(scale) +".csv")



[18:15:24] INFO: ../src/tree/updater_prune.cc:98: tree pruning end, 126 extra nodes, 0 pruned nodes, max_depth=6
[18:15:26] INFO: ../src/tree/updater_prune.cc:98: tree pruning end, 126 extra nodes, 0 pruned nodes, max_depth=6
[18:15:30] INFO: ../src/tree/updater_prune.cc:98: tree pruning end, 126 extra nodes, 0 pruned nodes, max_depth=6
[18:15:34] INFO: ../src/tree/updater_prune.cc:98: tree pruning end, 126 extra nodes, 0 pruned nodes, max_depth=6
[18:15:38] INFO: ../src/tree/updater_prune.cc:98: tree pruning end, 126 extra nodes, 0 pruned nodes, max_depth=6
[18:15:41] INFO: ../src/tree/updater_prune.cc:98: tree pruning end, 126 extra nodes, 0 pruned nodes, max_depth=6
[18:15:44] INFO: ../src/tree/updater_prune.cc:98: tree pruning end, 126 extra nodes, 0 pruned nodes, max_depth=6
[18:15:49] INFO: ../src/tree/updater_prune.cc:98: tree pruning end, 124 extra nodes, 0 pruned nodes, max_depth=6
[18:15:53] INFO: ../src/tree/updater_prune.cc:98: tree pruning end, 126 extra nodes, 0 pruned no